In [11]:
from grammar import Grammar
import grammar.old_grammar as old_grammar
from scipy import sparse
import constants as c
import numpy as np
import pandas as pd
from utils.trie import Trie
from  tqdm import tqdm

df = pd.read_csv('../datasources/IL_ESW_extended/anions.csv')


In [4]:
smiles = 'CCCCC[InH-](CCCN)N'

dense_ohe = old_grammar.parse_smiles_list([smiles])[0]
print(dense_ohe.shape)
dense_ohe[0]

ohe = sparse.coo_array(dense_ohe)


(494, 71)


In [6]:
def check_sparse_ohe_validity(ohe: sparse.coo_array):
    if not ohe.has_canonical_format:
        print("OHE doesn't have canonical format")
        return False

    if not np.all(ohe.row == range(c.max_length)):
        print("OHE doesn't have a single non zero by by line.")
        return False

    
    if not np.all(ohe.data == np.ones(c.max_length)):
        print("OHE have non zero value different than 1.")
        return False
    
    return True


In [10]:
display(df)

,Unnamed: 0,smiles,mol_id,canonical,homo-fopt,lumo-fopt,gap-fopt,e_tot-fopt
0,0,FC(C(F)(C(F)(F)C(F)(F)C(F)(F)F)F)(F)C(F)(F)CC[...,a-0,CC(C[N-]c1ccc(cc1)[Si](CCC(C(C(C(C(C(F)(F)F)(F...,-0.111972,0.764155,0.876126,-65858.666435
1,1,C(F)(C(F)(F)C(C(F)(CC[Si](C)(C)c1ccc([N-]CC(C)...,a-0,CC(C[N-]c1ccc(cc1)[Si](CCC(C(C(C(C(C(F)(F)F)(F...,-0.111972,0.764155,0.876126,-65858.666435
2,2,C(F)(F)(C(C(CC[Si](C)(c1ccc([N-]CC(C)C)cc1)C)(...,a-0,CC(C[N-]c1ccc(cc1)[Si](CCC(C(C(C(C(C(F)(F)F)(F...,-0.111972,0.764155,0.876126,-65858.666435
3,3,C[Si](CCC(F)(C(C(F)(F)C(F)(F)C(F)(C(F)(F)F)F)(...,a-0,CC(C[N-]c1ccc(cc1)[Si](CCC(C(C(C(C(C(F)(F)F)(F...,-0.111972,0.764155,0.876126,-65858.666435
4,4,CC(C)C[N-]c1ccc([Si](C)(C)CCC(F)(F)C(C(C(C(F)(...,a-0,CC(C[N-]c1ccc(cc1)[Si](CCC(C(C(C(C(C(F)(F)F)(F...,-0.111972,0.764155,0.876126,-65858.666435
...,...,...,...,...,...,...,...,...
3207268,941,C(C(F)(F)F)(F)(C(F)(C(F)(F)C(C(F)(F)c1ccc([N-]...,a-7443,Br[N-]c1ccc(cc1)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(...,-0.628117,1.259612,1.887729,-120755.899233
3207269,942,FC(C(F)(C(c1ccc([N-]Br)cc1)(F)F)F)(C(F)(C(C(F)...,a-7443,Br[N-]c1ccc(cc1)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(...,-0.628117,1.259612,1.887729,-120755.899233
3207270,943,Br[N-]c1ccc(C(F)(C(C(F)(F)C(C(C(F)(F)F)(F)F)(F...,a-7443,Br[N-]c1ccc(cc1)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(...,-0.628117,1.259612,1.887729,-120755.899233
3207271,944,c1([N-]Br)ccc(C(F)(F)C(F)(C(F)(F)C(F)(F)C(C(F)...,a-7443,Br[N-]c1ccc(cc1)C(C(C(C(C(C(F)(F)F)(F)F)(F)F)(...,-0.628117,1.259612,1.887729,-120755.899233


In [20]:
trie = Trie()

total = len(df.index)
progress_bar = tqdm(enumerate(df['smiles']), total=total)
count = 0
for i, smiles in progress_bar:
    dense_ohe = old_grammar.parse_smiles_list([smiles])[0]
    ohe = sparse.coo_array(dense_ohe)
    identifier = ohe.col
    if(not trie.has(identifier)):
        trie.set(identifier, 'smiles')
        count += 1
        progress_bar.set_description(f'Count: {count} ({count/(i + 1):.2%})')





  0%|          | 0/3207273 [00:00<?, ?it/s]

Count: 5456 (100.00%):   0%|          | 5456/3207273 [03:03<29:57:24, 29.69it/s]


KeyboardInterrupt: 